In [1]:
import tensorflow as tf
import numpy as np
import seaborn as sns
import mlflow
import dagshub
import json
import os
import keras
import matplotlib.pyplot as plt
from dagshub import dagshub_logger
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
from kerastuner.tuners import RandomSearch, BayesianOptimization

2025-05-28 11:31:14.361238: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-28 11:31:14.935569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748424675.142763     749 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748424675.197368     749 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748424675.666828     749 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Defining mlflow experiment parameter
TRIAL_NAME = "trial_"
MLFLOW_MAIN_RUN = "Main Tunining Run: Augmented Image Model "
TUNER_DIRECTORY = "logs/tuner"
TUNER_PROJECT_NAME = "FingerSpellIT - Augmented Image Model"
MODEL_NAME = "augmented_image_model_v1"
EVALUATION_MLFLOW_RUN = "Evaluating Augmented Image Model"
REPORT_NAME = "augmented_image_model_classification_report"
CM_NAME="augmented_image_model_confusion_matrix_best_model"

In [3]:
# Dagshub Initialization
mlflow.set_tracking_uri("https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow")

dagshub.init(repo_owner="alfoCaiazza", repo_name="FingerSpellIT", mlflow=True)
dagshub_log = dagshub_logger(metrics_path="metrics", hparams_path="params")

Accessing as alfoCaiazza

Initialized MLflow to track repo "alfoCaiazza/FingerSpellIT"

Repository alfoCaiazza/FingerSpellIT initialized!

In [4]:
# To avoid OOM errors, setting GPU Memory Consuption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(f"GPU: {gpu}")
    tf.config.experimental.set_memory_growth(gpu, True) # Keeping the use of memory limited to prevent errors

GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [5]:
train_dir = '../data/processed_imgs/splits/train'
test_dir = '../data/processed_imgs/splits/test'
val_dir = '../data/processed_imgs/splits/val'

# Creating dataset with keras method
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224,224),
    batch_size=32,
    shuffle=True,
    seed=123
)

class_names = train_ds.class_names

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224,224),
    batch_size=32,
    shuffle=True,
    seed=123
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224,224),
    batch_size=32,
    shuffle=True,
    seed=123
)



Found 28312 files belonging to 24 classes.


I0000 00:00:1748424698.320430     749 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1753 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Found 3546 files belonging to 24 classes.
Found 3532 files belonging to 24 classes.


In [6]:
# Pre-processing sequential model
# Normalizing the images for ResNet50 model (0,1)
normalization = tf.keras.layers.Rescaling(1./255)

train_ds  = train_ds.map(lambda x, y: (normalization(x), y))
val_ds = val_ds.map(lambda x, y: (normalization(x), y))
test_ds = test_ds.map(lambda x, y: (normalization(x), y))

In [7]:
# Handling data batch size automatically 
AUTOTUNE = tf.data.AUTOTUNE

train_ds= train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds= val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [8]:
#Building parametrized ResNet50 Transfer Learning Model
def build_model(hp):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224,224,3)
    )

    # Unfreezing last layers
    base_model.trainable = True
    for layer in base_model.layers[:-30]:
        layer.trainable = False

    activation = hp.Choice('activation', ['relu', 'sigmoid', 'tanh'])

    # Building output layers
    x = GlobalAveragePooling2D()(base_model.output)
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        x = Dense(hp.Int('num_units', 128, 512, step=128))(x)
        x = Dropout(hp.Float('dropout_rate', 0.2, 0.5))(x)

    predictions = Dense(24, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Parametrizing optimizers
    optimizer_name = hp.Choice('optimizer', ['Adam', 'RMSprop', 'SGD'])

    if optimizer_name == 'Adam':
        optimizer = Adam(
            learning_rate = hp.Float('adam_lr', min_value=1e-5, max_value=1e-2, sampling='log')
    )
    elif optimizer_name == 'RMSprop':
        optimizer = RMSprop(
               learning_rate = hp.Float('rmsprop_lr', min_value=1e-5, max_value=1e-2, sampling='log'),
               rho = hp.Float('rmsprop_rho', min_value=0.8, max_value=0.99)
    )
    elif optimizer_name == 'SGD':
            optimizer = SGD(
                learning_rate=hp.Float('sgd_lr', min_value=1e-4, max_value=1e-2, sampling='log'),
                momentum=hp.Float('sgd_momentum', min_value=0.0, max_value=0.99)
    )
            
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics = ['sparse_categorical_accuracy']
    )

    return model

In [9]:
base_callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-5)
]

# Tracking MLflow Callback
class MLflowCallback(Callback):
    def __init__(self, trial_hyperparameters, trial_id):
        super().__init__()
        self.trial_id = trial_id
        self.trial_hyperparameters =trial_hyperparameters

    def on_train_begin(self, logs=None):
        self.run = mlflow.start_run(run_name=f"{TRIAL_NAME}_{self.trial_id}", nested=True)

        # Tracks trial parameters
        mlflow.log_param('trial_id', self.trial_id)
        for param_name, param_value in self.trial_hyperparameters.values.items():
            mlflow.log_param(param_name, param_value)

    # Tracks trial metrics
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for metric_name, value in logs.items():
                mlflow.log_metric(metric_name, value, step=epoch)

    # Close trial mlrun 
    def on_train_end(self, logs=None):
        if self.run:
            mlflow.end_run()



In [10]:
# Sublassed RandomSearch Tuner that implements MLflow tracking
class MLflowTuner(BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        callbacks = base_callbacks + [MLflowCallback(trial.hyperparameters, trial.trial_id)]
        kwargs['callbacks'] = callbacks

        return super().run_trial(trial, *args, **kwargs)

In [ ]:
with mlflow.start_run(run_name=f"{MLFLOW_MAIN_RUN}"):
    epochs = 50

    mlflow.set_tag("stage", "hyperparameter_tuning")
    mlflow.set_tag("model_architecture", "ResNet50")
    mlflow.log_param('epochs', epochs)

    tuner = MLflowTuner(
        build_model,
        objective='sparse_categorical_accuracy',
        max_trials=5,
        executions_per_trial=1,
        directory=f'{TUNER_DIRECTORY}',
        project_name=f'{TUNER_PROJECT_NAME}'
    )
    
    tuner.search(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
    )

    best_model = tuner.get_best_models(num_models=1)[0]
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

    for param, value in best_hps.values.items():
        mlflow.log_param(param, value)

    # Log metriche finali del trial
    for metric_name, metric_data in best_trial.metrics.metrics.items():
        if metric_data and isinstance(metric_data, dict):
            values = metric_data.get('value', [])
            if values:
                mlflow.log_metric(metric_name, values[-1])

    # Log best score
    mlflow.log_metric("best_sparse_categorical_accuracy", best_trial.score)

    # Salving the model
    model_path = f"../models/{MODEL_NAME}.keras"
    best_model.save(model_path)
    mlflow.log_artifact(model_path)

mlflow.end_run()

Trial 5 Complete [00h 06m 00s]
sparse_categorical_accuracy: 1.0

Best sparse_categorical_accuracy So Far: 1.0
Total elapsed time: 00h 49m 06s


/home/acaia/FingerSpellIT/FingerSpellIT/.fingerspellit/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 82 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


🏃 View run Main Tunining Run: In-Vitro Image Model  at: https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow/#/experiments/0/runs/bcdad5cf4fcc4f28b4fa8df501066216
🧪 View experiment at: https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow/#/experiments/0


In [8]:
best_model = tf.keras.models.load_model('../models/augmented_image_model_v1.keras')

/home/acaia/FingerSpellIT/FingerSpellIT/.fingerspellit/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 82 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
test_loss, test_acc = best_model.evaluate(test_ds)
print(f"Evaluation accuracy: {test_acc:.3f}")

y_true, y_pred = [], []

for images, labels in test_ds:
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(best_model.predict(images), axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

I0000 00:00:1748424765.159218    1087 service.cc:152] XLA service 0x7f73e484af80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748424765.159431    1087 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2025-05-28 11:32:46.112320: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748424767.133442    1087 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-28 11:32:49.267265: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1698', 204 bytes spill stores, 204 bytes spill loads

2025-05-28 11:32:49.633617: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_

  1/111 ━━━━━━━━━━━━━━━━━━━━ 30:40 17s/step - loss: 0.0763 - sparse_categorical_accuracy: 0.9688

I0000 00:00:1748424778.298602    1087 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


110/111 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.1269 - sparse_categorical_accuracy: 0.9660

2025-05-28 11:33:15.623211: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1698', 4 bytes spill stores, 4 bytes spill loads

2025-05-28 11:33:16.344449: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1698', 220 bytes spill stores, 220 bytes spill loads

2025-05-28 11:33:16.555122: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1698', 8 bytes spill stores, 8 bytes spill loads

2025-05-28 11:33:16.776258: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1698', 4384 bytes spill stores, 4228 bytes spill loads

2025-05-28 11:33:16.929339: I external

111/111 ━━━━━━━━━━━━━━━━━━━━ 41s 224ms/step - loss: 0.1273 - sparse_categorical_accuracy: 0.9660
Evaluation accuracy: 0.962
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━

2025-05-28 11:34:00.348820: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
# Parameters configuration
output_dir = "../artifacts/augmented_img"
report_name = REPORT_NAME 
cm_name = CM_NAME          
run_name = EVALUATION_MLFLOW_RUN

os.makedirs(output_dir, exist_ok=True)

with mlflow.start_run(run_name=run_name):
    # Classification report as a JSON file
    report_json_path = os.path.join(output_dir, f"{report_name}.json")
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    print("Classification Report", json.dumps(report, indent=4))

    with open(report_json_path, "w") as f:
        json.dump(report, f, indent=4)

    # Classification report as a txt file
    report_txt_path = os.path.join(output_dir, f"{report_name}.txt")
    with open(report_txt_path, "w") as f:
        f.write(classification_report(y_true, y_pred, target_names=class_names))

    # Log on MLflow
    mlflow.log_artifact(report_json_path, "evaluation_metrics")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()

    cm_path = os.path.join(output_dir, f"{cm_name}.png")
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')

    # Log dell'immagine su MLflow
    mlflow.log_figure(plt.gcf(), f"evaluation_plots/{cm_name}.png")
    plt.close()


Classification Report {
    "a": {
        "precision": 0.9603174603174603,
        "recall": 0.983739837398374,
        "f1-score": 0.9718875502008032,
        "support": 123.0
    },
    "b": {
        "precision": 0.7150837988826816,
        "recall": 1.0,
        "f1-score": 0.8338762214983714,
        "support": 128.0
    },
    "c": {
        "precision": 0.984251968503937,
        "recall": 0.984251968503937,
        "f1-score": 0.984251968503937,
        "support": 127.0
    },
    "d": {
        "precision": 1.0,
        "recall": 0.927007299270073,
        "f1-score": 0.9621212121212122,
        "support": 137.0
    },
    "e": {
        "precision": 1.0,
        "recall": 0.9933333333333333,
        "f1-score": 0.9966555183946488,
        "support": 150.0
    },
    "f": {
        "precision": 0.9702970297029703,
        "recall": 0.6950354609929078,
        "f1-score": 0.8099173553719008,
        "support": 141.0
    },
    "g": {
        "precision": 1.0,
        "recall":